In [2]:
import tensorflow as tf
import numpy as np
from transformers import TFBertModel

2022-12-09 20:45:42.484024: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 20:45:42.602749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 20:45:42.602772: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-09 20:45:42.632556: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-09 20:45:43.318072: W tensorflow/stream_executor/platform/de

In [3]:
import os, sys, re

In [8]:
ROOT = os.path.realpath("..")
print(ROOT)

/home/ubuntu/tempus_fugit/flatblack/benchmarking/tensorflow


In [19]:
# Set model environment variables

MODEL_NAME = 'bert-base-uncased'
MODEL_BASE_PATH = f'{ROOT}/models/bert/'

os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_NAME_UNDER'] = re.sub("-","_", MODEL_NAME)
os.environ['MODEL_BASE_PATH'] = MODEL_BASE_PATH

! echo $MODEL_NAME $MODEL_NAME_UNDER $MODEL_BASE_PATH 
! cd $MODEL_BASE_PATH && ls
! echo "export MODEL_BASE_PATH=$MODEL_BASE_PATH" >> ~/.bash_profile


bert-base-uncased bert_base_uncased /home/ubuntu/tempus_fugit/flatblack/benchmarking/tensorflow/models/bert/
assets	saved_model.pb	variables


In [11]:
# Download example model - in this case bert-base-uncased
model = TFBertModel.from_pretrained(MODEL_NAME)
# Save model in SavedModel Format
tf.saved_model.save(model,MODEL_BASE_PATH)

2022-12-09 20:48:50.788328: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-09 20:48:50.788360: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-09 20:48:50.788381: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-25-33): /proc/driver/nvidia/version does not exist
2022-12-09 20:48:50.788642: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['n

INFO:tensorflow:Assets written to: /home/ubuntu/tempus_fugit/flatblack/benchmarking/tensorflow/models/bert/assets


INFO:tensorflow:Assets written to: /home/ubuntu/tempus_fugit/flatblack/benchmarking/tensorflow/models/bert/assets


In [31]:
# Build a tensorflow-serve docker image
#! docker pull tensorflow/serving
#! docker images | grep tensorflow/serving
! docker run -d --name tf_serving_base tensorflow/serving


Unable to find image 'tensorflow/serving:latest' locally
latest: Pulling from tensorflow/serving

e5416296: Pulling fs layer 
1bab055f: Pulling fs layer 
6ae4c4ea: Pulling fs layer 
66225230: Pulling fs layer 
Digest: sha256:e66c1866bf6596473d56beb3ef77bf7b8b00baa0e1e80c3c428ede31da8ae066[2A
Status: Downloaded newer image for tensorflow/serving:latest
aeb73c0d0a1b5d87ebb1135da1fffa77b6027cca9d38e9ed8b9acaae0974b202


In [34]:
! echo $MODEL_BASE_PATH

/home/ubuntu/tempus_fugit/flatblack/benchmarking/tensorflow/models/bert/


In [35]:
# Copy model into  tensorflow serving container
! cd $MODEL_BASE_PATH && ls
! docker cp $MODEL_BASE_PATH tf_serving_base:/models/$MODEL_NAME_UNDER

1


In [36]:
!echo $MODEL_BASE_PATH
!echo ENV MODEL_NAME ${MODEL_NAME}

/home/ubuntu/tempus_fugit/flatblack/benchmarking/tensorflow/models/bert/
ENV MODEL_NAME -base-uncased


In [37]:
# Commit the container to serving the model 
! docker commit --change "ENV MODEL_NAME ${MODEL_NAME_UNDER}" tf_serving_base $USER/tf_serving_base

sha256:8c7f9527d570a800257014b64978980cee2cd4461d154f7ddc0163b088d74170


In [38]:
# Kill tf_serving base 
! docker kill tf_serving_base
! docker rm tf_serving_base

tf_serving_base
tf_serving_base


In [ ]:
! docker images | grep $USER/tf_serving_base

# Start the container with our model
! docker run -p 8500:8500 -t $USER/tf_serving_base &

In [39]:
!docker run -p 8501:8501 \
    -v "${MODEL_BASE_PATH}:/models/${MODEL_NAME_UNDER}" \
    -e MODEL_NAME=$MODEL_NAME_UNDER \
    -t $USER/tf_serving_base

2022-12-09 22:28:33.267495: I tensorflow_serving/model_servers/server.cc:74] Building single TensorFlow model file config:  model_name: bert_base_uncased model_base_path: /models/bert_base_uncased
2022-12-09 22:28:33.267723: I tensorflow_serving/model_servers/server_core.cc:465] Adding/updating models.
2022-12-09 22:28:33.267748: I tensorflow_serving/model_servers/server_core.cc:594]  (Re-)adding model: bert_base_uncased
2022-12-09 22:28:33.459775: I tensorflow_serving/core/basic_manager.cc:739] Successfully reserved resources to load servable {name: bert_base_uncased version: 1}
2022-12-09 22:28:33.459822: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: bert_base_uncased version: 1}
2022-12-09 22:28:33.459843: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: bert_base_uncased version: 1}
2022-12-09 22:28:33.459892: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /models

In [22]:
! echo "${MODEL_BASE_PATH}:/models/${MODEL_NAME_UNDER}"

/home/ubuntu/tempus_fugit/flatblack/benchmarking/tensorflow/models/bert/:/models/bert_base_uncased
